In [2]:
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from jinja2.ext import Extension

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
df = pd.read_json('llava_instruct_150k_llavar_20k.json')
df

,id,image,conversations
0,33471,000000033471.jpg,"[{'from': 'human', 'value': '<image> What are ..."
1,52846,000000052846.jpg,"[{'from': 'human', 'value': 'Where is the cat ..."
2,334872,000000334872.jpg,"[{'from': 'human', 'value': '<image> Are the p..."
3,319154,000000319154.jpg,"[{'from': 'human', 'value': '<image> What colo..."
4,398214,000000398214.jpg,"[{'from': 'human', 'value': 'What type of sign..."
...,...,...,...
177507,100000027657,100000027657.jpg,"[{'from': 'human', 'value': 'What is the main ..."
177508,100001258985,100001258985.jpg,"[{'from': 'human', 'value': '<image> What is t..."
177509,100000288504,100000288504.jpg,"[{'from': 'human', 'value': 'What is the main ..."
177510,100000680633,100000680633.jpg,"[{'from': 'human', 'value': '<image> What does..."


In [5]:
tokenizer = AutoTokenizer.from_pretrained("anananan116/TinyVLM")

In [6]:
def conversation_tokenize(c):
    SYSTEM_PROMPT = "You are a powerful visual assistant."
    # Add the system prompt to the start of the conversation
    c = [{"role": "system", "content": SYSTEM_PROMPT}] + c
    c_transformed = []

    for message in c:
        transformed_message = {
            'role': message.get('from', message.get('role')),
            'content': message.get('value', message.get('content'))
        }
        # Replace <image> placeholder
        transformed_message['content'] = transformed_message['content'].replace('<image>', '<IMGPLH>')
        # Map roles
        if transformed_message['role'] == 'human':
            transformed_message['role'] = 'user'
        elif transformed_message['role'] == 'gpt':
            transformed_message['role'] = 'assistant'
        c_transformed.append(transformed_message)

    return c_transformed

In [7]:
df['conversations']=df['conversations'].apply(conversation_tokenize)
df

,id,image,conversations
0,33471,000000033471.jpg,"[{'role': 'system', 'content': 'You are a powe..."
1,52846,000000052846.jpg,"[{'role': 'system', 'content': 'You are a powe..."
2,334872,000000334872.jpg,"[{'role': 'system', 'content': 'You are a powe..."
3,319154,000000319154.jpg,"[{'role': 'system', 'content': 'You are a powe..."
4,398214,000000398214.jpg,"[{'role': 'system', 'content': 'You are a powe..."
...,...,...,...
177507,100000027657,100000027657.jpg,"[{'role': 'system', 'content': 'You are a powe..."
177508,100001258985,100001258985.jpg,"[{'role': 'system', 'content': 'You are a powe..."
177509,100000288504,100000288504.jpg,"[{'role': 'system', 'content': 'You are a powe..."
177510,100000680633,100000680633.jpg,"[{'role': 'system', 'content': 'You are a powe..."


In [8]:
df.to_csv('170k_data_tokenized.csv', index=False)